In [7]:
import cv2 as cv
import numpy as np
from cv2 import aruco
import math


In [8]:
def detect_ArUco_details(image):
    ArUco_details_dict = {}
    ArUco_corners = {}

    ##############	ADD YOUR CODE HERE	##############
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image_gray = cv.GaussianBlur(image_gray, (3, 3), 1)
    dictionary = cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_1000)
    parameters = cv.aruco.DetectorParameters()
    detector = cv.aruco.ArucoDetector(dictionary, parameters)
    bboxs, ids, _ = detector.detectMarkers(image)
    # image = cv2.flip(image, 1)

    if ids is not None:
        # bboxs[0][0][3],bboxs[0][0][2] = bboxs[0][0][2],bboxs[0][0][3]
        # for i in range(0, 3):
        #     bboxs[0][0][i][0] = abs(bboxs[0][0][i][0] - frame_width)
        ArUco_corners2 = (np.int0(bboxs)).ravel()
        # print(bboxs[0][0][0][0])
        ArUco_corners = {}
        ArUco_corners_list = []
        count1 = 0
        count2 = 0
        for i in range(0, len(ArUco_corners2), +2):
            temp = []
            temp.append(ArUco_corners2[i])
            temp.append(ArUco_corners2[i + 1])
            count1 += 1
            ArUco_corners_list.append(temp)
            if count1 % 4 == 0 and count1 != 0:
                ArUco_corners[count2] = ArUco_corners_list
                count2 += 1
                ArUco_corners_list = []
        ArUco_ID = ids.ravel()
        centre = []
        for i in range(0, len(ArUco_corners)):
            sumx = 0
            sumy = 0
            for j in range(0, 4):
                sumx += ArUco_corners[i][j][0]
                sumy += ArUco_corners[i][j][1]
            lst = []
            lst.append(int(sumx / 4.0))
            lst.append(int(sumy / 4.0))
            centre.append(lst)

        slope = []
        for i in range(0, len(ArUco_corners)):
            vr = 0
            x1 = int(round((ArUco_corners[i][0][0] + ArUco_corners[i][1][0]) / 2))
            y1 = int(round((ArUco_corners[i][0][1] + ArUco_corners[i][1][1]) / 2))
            x2 = centre[i][0]
            y2 = centre[i][1]
            c = 0
            if x2 == x1 and y2 <= y1:  # 2nd Quadrant
                vr = -180
                c += 1
            elif x2 <= x1 and y2 == y1:  # 1st Quadrant
                vr = -90
                c += 1
            elif x2 >= x1 and y2 == y1:  # 3rd Quadrant
                vr = 90
                c += 1
            elif x2 == x1 and y2 >= y1:  # 4th Quadrant
                vr = 0
                c += 1
            if c == 0:
                x1 = ArUco_corners[i][0][0]
                x2 = ArUco_corners[i][1][0]
                y1 = ArUco_corners[i][0][1]
                y2 = ArUco_corners[i][1][1]
                if x1 != x2:
                    vr = int(round(math.degrees(math.atan2((y1 - y2), (x2 - x1)))))
                vr = int(vr)
                x1 = int(round((ArUco_corners[i][0][0] + ArUco_corners[i][1][0]) / 2))
                y1 = int(round((ArUco_corners[i][0][1] + ArUco_corners[i][1][1]) / 2))
                x2 = centre[i][0]
                y2 = centre[i][1]
            slope.append(vr)
        lst = []
        for i in range(0, len(ArUco_ID)):
            lst.append(centre[i])
            lst.append(int(slope[i]))
            ArUco_details_dict[int(ArUco_ID[i])] = lst
            lst = []
        ArUco_corners_copy = ArUco_corners.copy()
        ArUco_corners = {}
        for i in range(0, len(ArUco_ID)):
            ArUco_corners[int(abs(ArUco_ID[i]))] = ArUco_corners_copy[i]

    return ArUco_details_dict, ArUco_corners


### Camera matrix and Distortion coefficients

In [9]:
# import glob

# # termination criteria
# criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# objp = np.zeros((6*7,3), np.float32)
# objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# # Arrays to store object points and image points from all the images.
# objpoints = [] # 3d point in real world space
# imgpoints = [] # 2d points in image plane.

# images = glob.glob('calibration_images/*.jpg')

# for fname in images:
#     img = cv.imread(fname)
#     gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

#     # Find the chess board corners
#     ret, corners = cv.findChessboardCorners(gray, (7,6), None)

#     # If found, add object points, image points (after refining them)
#     if ret == True:
#         objpoints.append(objp)

#         corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
#         imgpoints.append(corners2)

#         # Draw and display the corners
#         img = cv.drawChessboardCorners(img, (7,6), corners2, ret)
#         cv.imshow('img', img)
#         cv.waitKey(500)

# cv.destroyAllWindows()

# # Calibrate the camera
# ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# print("Camera matrix : \n")
# print(mtx)
# print("dist : \n")
# print(dist)


In [10]:
def event_identification(arena):
    event_list = []
    arena_org = arena.copy()
    image_cnts = 0
    border_width = 15
    arena_gray = cv.GaussianBlur(arena, (3, 3), 1)
    arena_gray = cv.cvtColor(arena_gray, cv.COLOR_BGR2GRAY)
    cv.imshow("gray", arena_gray)
    ret, thresh = cv.threshold(arena_gray, 200, 255, cv.THRESH_BINARY)
    cv.imshow("thresh", thresh)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
        if len(approx) == 4:
            x1, y1, w, h = cv.boundingRect(cnt)
            ratio = float(w) / h
            area = w * h
            # print("approx", approx[0])
            if ratio >= 0.8 and ratio <= 1.3:
                # if ratio >= 0.9 and ratio <= 1.1 and area > 3000 and area < 15000:
                # cropped_image = arena_org[
                #     y1 + border_width : y1 + 120 - border_width,
                #     x1 + border_width : x1 + 120 - border_width,
                # ]
                # cv.imshow("CI",cropped_image)
                
                print("Area at ", image_cnts, " is ", area)
                image_cnts += 1
                list = []
                for i in range(0, 4):
                    list.append(approx[i][0])
                event_list.append(list)
                arena = cv.rectangle(arena, (event_list[0][0],event_list[0][1]), (event_list[0][0]+event_list[0][2],event_list[0][1]+event_list[0][3]), (0,255,0), 3)
                print("event_list",event_list[0][0])
    cv.imshow("Img",arena)
    cv.waitKey(0)
    cv.destroyAllWindows()
    return event_list, image_cnts


In [11]:
#event_identification(cv.imread("/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4A/ROI_29.jpg"))


In [12]:
import cv2 as cv
import tkinter as tk
from tkinter import ttk
import numpy as np


# img =cv.imread("/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Arena.png")
# event_list , image_cnts = event_identification(img)
# print("event_list",event_list)
# print("image_cnts",image_cnts)
# for i in range(0,image_cnts):
#     image = cv.rectangle(img, (event_list[i][0],event_list[i][1]), (event_list[i][0]+event_list[i][2],event_list[i][1]+event_list[i][3]), (0,255,0), 3)
# cv.imshow("window_name", image)
# cv.waitKey(0)
# cv.destroyAllWindows()


try:
    cap = cv.VideoCapture("/dev/video2")
    if not cap.isOpened():
        raise Exception("Could not open /dev/video2")
except:
    cap = cv.VideoCapture("/dev/video3")
    if not cap.isOpened():
        print("Could not open /dev/video3 either. Please check your device.")
        try:
            cap = cv.VideoCapture("/dev/video0")
            if not cap.isOpened():
                raise Exception("Could not open /dev/video0")
        except:
            cap = cv.VideoCapture("/dev/video1")
            if not cap.isOpened():
                print("Could not open /dev/video1 either. Please check your device.")
desired_width = 1920
desired_height = 1080
cap.set(cv.CAP_PROP_FRAME_WIDTH, desired_width)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, desired_height)
event_arcs = [21, 30, 34, 48]
# roi = []
while 1:
    # Capture the video frame
    # by frame
    frame = cv.imread(
        "/mnt/Storage/Pictures/picture_2023-12-23_23-59-36 (copy).jpg"
    )
    # ret, frame = cap.read()
    # frame = cv.rotate(frame, cv.ROTATE_90_COUNTERCLOCKWISE)
    ArUco_details_dict, _ = detect_ArUco_details(frame)
    # event_list, image_cnts = event_identification(frame)
    for i in event_arcs:
        if ArUco_details_dict[i][1] > -100 and ArUco_details_dict[i][1] < -80:
            point1 = (ArUco_details_dict[i][0][0] + 50, ArUco_details_dict[i][0][1])
            point2 = (
                ArUco_details_dict[i][0][0] - 120,
                ArUco_details_dict[i][0][1] - 200,
            )
            top_left = (min(point1[0], point2[0]), min(point1[1], point2[1]))
            bottom_right = (max(point1[0], point2[0]), max(point1[1], point2[1]))
            roi = frame[top_left[1] : bottom_right[1], top_left[0] : bottom_right[0]]
        elif ArUco_details_dict[i][1] < 10 and ArUco_details_dict[i][1] > -10:
            roi = frame[
                (ArUco_details_dict[i][0][0], ArUco_details_dict[i][0][1] - 50),
                (ArUco_details_dict[i][0][0] - 200, ArUco_details_dict[i][0][1] + 120),
            ]
        cv.imshow("ROI_" + str(i), roi)
        cv.imwrite(
            "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4A/ROI_"
            + str(i)
            + ".jpg",
            roi,
        )
        flag = 0
        for l_thresh in range(3, 52, +2):
            arena_gray = cv.GaussianBlur(roi, (3, 3), 0)
            arena_gray = cv.cvtColor(arena_gray, cv.COLOR_BGR2GRAY)
            # cv.imshow("gray", arena_gray)
            # ret, thresh = cv.threshold(arena_gray, l_thresh, 256, cv.THRESH_BINARY)
            thresh = cv.adaptiveThreshold(
                arena_gray, 240, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, l_thresh, 1
            )
            contours, hierarchy = cv.findContours(
                thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE
            )
            for cnt in contours:
                approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
                if len(approx) == 4:
                    x1, y1, w, h = cv.boundingRect(cnt)
                    ratio = float(w) / h
                    area = w * h
                    # print("approx", approx[0])
                    if ratio >= 0.9 and ratio <= 1.2 and area > 7000 and area < 9800:
                        list = []
                        for j in range(0, 4):
                            list.append(approx[j][0])
                        event_list = [list]
                        flag = 1
            if flag == 1:
                break
        print("Thresh for ", str(i), " = ", l_thresh)
        arena_gray = cv.GaussianBlur(roi, (3, 3), 1)
        arena_gray = cv.cvtColor(arena_gray, cv.COLOR_BGR2GRAY)
        ret, thresh = cv.threshold(arena_gray, l_thresh, 255, cv.THRESH_BINARY)
        cv.imshow("thresh" + str(i), thresh)

        # pts = np.reshape(event_list[i], (-1, 1, 2))
        # # print("pts", event_list[i])
        # width = 90
        # height = 90
        # # for i in pts:
        # #     frame = cv.circle(
        # #         frame, (i[0][0], i[0][1]), radius=0, color=(0, 255, 0), thickness=10
        # #     )
        # # intersect_pts = list()
        # # for j in pts:
        # #     lst = list()
        # #     lst.append(i[0][0])
        # #     lst.append(i[0][1])
        # #     intersect_pts.append(i)
        # dstPts = [[0, 0], [width, 0], [width, height], [0, height]]
        # # print("intersect_pts", pts)
        # # Get the transform
        # matrix = cv.getPerspectiveTransform(np.float32(pts), np.float32(dstPts))
        # # Transform the image
        # out = cv.warpPerspective(frame, matrix, (int(width), int(height)))
        # # out = cv.flip(out, 1)
        # image_path = (
        #     "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4A/Saved_Images/Img_"
        #     + str(i)
        #     + ".jpg"
        # )
        # cv.imwrite(image_path, out)
        # image = cv.polylines(frame, [pts], 1, (0, 255, 0), 3)

    # image = cv.resize(image, (1900, 900))
    # cv.imshow("Image", image)
    # print("Out", out.shape)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    # Display the resulting frame

    # cv.imshow('frame', frame)

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv.destroyAllWindows()


/tmp/ipykernel_4352/974757838.py:18: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  ArUco_corners2 = (np.int0(bboxs)).ravel()
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  48  =  3
Thresh for  21  =  3
Thresh for  30  =  3
Thresh for  34  =  3
Thresh for  4